In [3]:
import re
import pickle
from janome.tokenizer import Tokenizer
import collections
from gensim.models import word2vec
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


with open("bot.txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    bot_original = f.read()

bot = re.sub("[\n]", "。", bot_original) # | と全角半角スペース、「」と改行の削除
bot = re.sub("[｜ 　]", "", bot) # | と全角半角スペース、「」と改行の削除

# 終わり記号の重複を除去
bot = bot.replace('！。', '！')
bot = bot.replace('？。', '？')
bot = bot.replace('♪。', '♪')
bot = bot.replace('？*', '？')
bot = bot.replace('♪？', '♪')
bot = bot.replace('。*', '。')

seperator = "。"  # 。をセパレータに指定
bot_list = re.split('[。！？♪]', bot)  # セパレーターを使って文章をリストに分割する
bot_list.pop() # 最後の要素は空の文字列になるので、削除
bot_list.pop(0)
bot_list = [x+seperator for x in bot_list]  # 文章の最後に。を追加

t = Tokenizer()

bot_words = []
for sentence in bot_list:
    bot_words.append(t.tokenize(sentence, wakati=True))   # 文章ごとに単語に分割し、リストに格納
    
with open('bot_words.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(bot_words, f)

# 読み込み用
# with open('bot_words.pickle', mode='rb') as f:
#     bot_words = pickle.load(f)

NameError: name 'words' is not defined

In [8]:
# size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(bot_words,
                          size=100,
                          min_count=5,
                          window=5,
                          iter=20,
                          sg = 0)


In [9]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現
print(len(model.wv.index2word))  # 語彙の数
print(model.wv.index2word[:10])  # 最初の10単語

(2266, 100)
[[-3.6608776e-01  6.9012302e-01  4.0331030e-01 ...  3.3114445e-01
   8.5808325e-01 -2.0029101e-01]
 [-2.5216025e-01  1.3797539e-01  7.4951988e-01 ... -7.6588849e-03
  -3.3250514e-01 -1.5308677e-03]
 [-5.4016936e-01  2.2724479e-01  1.0155753e+00 ... -7.2973377e-01
   6.4893931e-01 -2.9359201e-01]
 ...
 [ 9.5886040e-05 -9.9446207e-02  7.1058877e-02 ...  8.4064431e-02
  -2.0338913e-02 -3.4190239e-03]
 [-6.0925116e-03 -1.2996139e-01  9.9249281e-02 ...  1.1599533e-01
  -5.2975252e-02  7.3325372e-04]
 [ 6.7871367e-04  3.4442514e-02 -4.7413964e-02 ...  1.6589938e-01
  -2.0105366e-02  1.4117502e-01]]
2266
['。', 'ばい', 'て', 'た', 'に', 'の', 'は', 'が', 'ね', 'あなた']


In [13]:
print(model.wv.most_similar("おはよう"))  # 最も似ている単語

[('なるほど', 0.9403193593025208), ('お疲れ様', 0.9233492016792297), ('おやすみ', 0.9017361402511597), ('了解', 0.8463051319122314), ('おめでとう', 0.8451974987983704), ('育休', 0.8440346717834473), ('かえる', 0.8137755393981934), ('すい', 0.8121225833892822), ('恥ずかしい', 0.8101416230201721), ('暖かい', 0.8036526441574097)]


In [15]:
tagged_documents = []
for i, sentence in enumerate(bot_words):
    tagged_documents.append(TaggedDocument(sentence, [i]))  # TaggedDocument型のオブジェクトをリストに格納

# size：分散表現の次元数
# window：対象単語を中心とした前後の単語数
# min_count：学習に使う単語の最低出現回数
# epochs:epochs数
# dm：学習モデル=DBOW（デフォルトはdm=1で、学習モデルはDM）
model = Doc2Vec(documents=tagged_documents,
                vector_size=100,
                min_count=5,
                window=5,
                epochs=20,
                dm=0)

In [16]:
print(bot_words[0])  # 最初の文章を表示
print(model.docvecs[0])  # 最初の文章のベクトル

['その', 'せい', 'で', '時間', 'なくなっ', 'て', 'ごめん', '。']
[-0.11868197 -0.07130735 -0.01238266 -0.1575949  -0.09459867  0.01884246
  0.01129084 -0.04596738 -0.07008135  0.00062519  0.1036119  -0.08251083
 -0.02306661  0.06548497  0.02011882  0.08888667  0.05701245  0.05906919
  0.17237821 -0.06243692  0.20753194 -0.126915   -0.04976975 -0.00107312
  0.00308469  0.10076164  0.08838043  0.0140352  -0.03272156 -0.11259116
 -0.05104002 -0.02882503 -0.05904601  0.03438864  0.01216092 -0.05625768
  0.02501594 -0.05520113 -0.06955463  0.05748038  0.08389363 -0.08089931
 -0.01539682 -0.06214973 -0.00212878  0.08251083 -0.0352633   0.01160701
 -0.22429672  0.11489253 -0.00280632  0.07531861  0.07710977  0.14089033
  0.08455271 -0.0330891   0.09535205 -0.03515467 -0.06135472 -0.08331118
  0.06049691  0.02695638 -0.05867619  0.10592671 -0.02791326  0.09465496
  0.09406038 -0.06744692 -0.03308775 -0.10167576  0.04034194 -0.16555281
  0.00226399  0.06903473  0.07158112 -0.00146238  0.00131955  0.05770475
 -

In [17]:
for p in model.docvecs.most_similar(0):
    print(bot_words[p[0]])

['スケジュール', '管理', 'でき', 'て', 'ない', '会社', 'が', '責任', '取る', 'べき', 'ばい', '。']
['用事', '済ん', 'だら', '教え', 'て', '。']
['徒歩', '35', '分', 'ばい', '。']
['爽やか', 'スマイル', 'で', '好', '印象', '与え', 'て', 'いく', '。']
['本格', '的', 'に', '引っ越し', 'の', '準備', 'し', 'てる', '。']
['まあ', '許し', 'て', 'あげる', 'ばい', '。']
['め', 'ま', '坊', 'ばい', '。']
['豚カツ', 'おいしい', '。']
['セブ', 'は', 'い', 'つ', '出発', '。']
['明け', 'まし', 'て', 'おめでとう', '。']
